In [1]:
# We are going to emulate that we operate in the future market with 1 contract long and 1 contract short but in a independent way

# we need to know
#                    1. Contract = ES
#                    2. Timetable = in 09:00   out 18:00
#                    3. Range-Trail = calculated in previous step
#                    4. Operation with order Stop Trail

In [1]:
import pandas as pd
import numpy as np
import datetime 
from datetime import date
import nbimporter                                #library to import functions defined in other jupyter notebook but use here 
from P02RangetrialEstimation import int2date, locate_1st_day, locate_lst_day

Importing Jupyter notebook from P02RangetrialEstimation.ipynb


In [3]:
int2date(20201005)

datetime.date(2020, 10, 5)

In [2]:
# import the dataframe

df_es_00=pd.read_csv('../Data/ES_00.csv',dtype={'date': str,'time': str,'open': float ,'max' : float ,'min' : float ,'close' : float ,'volume' : int,
                                              'datetime_US' : str, 'datetime_EU' : str, }, parse_dates=['datetime_US', 'datetime_EU'])
df_es_02=pd.read_csv('../Data/ES_02.csv',dtype={'date': str,'timetable': str,'max' : float ,'min' : float ,'range' : float ,'range-trail' : float,
                                              'range-avg' : float}, parse_dates=['date'])

In [3]:
df_es_00.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241774 entries, 0 to 241773
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   date         241774 non-null  object        
 1   time         241774 non-null  object        
 2   open         241774 non-null  float64       
 3   max          241774 non-null  float64       
 4   min          241774 non-null  float64       
 5   close        241774 non-null  float64       
 6   volume       241774 non-null  int64         
 7   datetime_US  241774 non-null  datetime64[ns]
 8   datetime_EU  241774 non-null  datetime64[ns]
dtypes: datetime64[ns](2), float64(4), int64(1), object(2)
memory usage: 16.6+ MB


In [11]:
df_es_02['date'][df_es_02[df_es_02['range-trail']!=0].index[0]].year*10000+ \
df_es_02['date'][df_es_02[df_es_02['range-trail']!=0].index[0]].month*100+ \
df_es_02['date'][df_es_02[df_es_02['range-trail']!=0].index[0]].day

20140103

In [5]:
df_es_02

,date,timetable,max,min,range,range-trail,range-avg
0,2009-09-28,09:00-18:30,1060.25,1037.00,23.25,0.0,0.000000
1,2009-09-29,09:00-18:30,1065.75,1053.25,12.50,0.0,0.000000
2,2009-09-30,09:00-18:30,1063.25,1041.50,21.75,0.0,0.000000
3,2009-10-01,09:00-18:30,1055.25,1031.50,23.75,0.0,0.000000
4,2009-10-02,09:00-18:30,1027.00,1012.00,15.00,0.0,0.000000
...,...,...,...,...,...,...,...
2554,2019-09-02,09:00-18:30,2926.50,2889.00,37.50,13.0,19.223518
2555,2019-09-03,09:00-18:30,2914.50,2891.00,23.50,13.0,19.223518
2556,2019-09-04,09:00-18:30,2936.25,2921.50,14.75,13.0,19.223518
2557,2019-09-05,09:00-18:30,2986.50,2950.50,36.00,13.0,19.223518


In [12]:
def market_operation (df_int, df_day, timetable_ini, timetable_end):
    """
    This function complete a dataframe df_day with open, close-long, close-short for a daily base in a especific timetable
    You can realise that we are going to operate long and short.
    On the other hand we will use the order STOP-TRAIL
    -Parameters
    ----------
    df_int : dataframe with original data (intraday movements of a future contract) ['date', 'time', 'open', 'max', 'min', 'close', 'volume']
    df_day : new dataframe to create with columns ['date', 'timetable', 'max', 'min', 'range', range-avg, range-trail]. We will add the columns
             ['open', 'close-long', 'close-short', 'close', long-rst, short-rst, long-acc, short-acc]
    timetable_ini : time the operation start everyday. Example='09:00'
    timetable_end : time the operation finish everyday. Example='18:00' 

    -Returns
    -------
    nothing
    The function fill the dataframe df_day with daily ['open', 'long-close', 'short-close', 'close', 'long-rst', 'short-rst', 'long-acc', 'short-acc']
    """
    
    pd.options.mode.chained_assignment = None                     # default='warn' This is to avoid warnings of chained-assignment
    
    # Create new columns in the dataframe
    df_day['open']=np.zeros(len(df_day)).astype('float')
    df_day['long-close']=np.zeros(len(df_day)).astype('float')
    df_day['short-close']=np.zeros(len(df_day)).astype('float')
    df_day['close']=np.zeros(len(df_day)).astype('float')
    df_day['long-rst']=np.zeros(len(df_day)).astype('float')
    df_day['short-rst']=np.zeros(len(df_day)).astype('float')
    df_day['long-acc']=np.zeros(len(df_day)).astype('float')
    df_day['short-acc']=np.zeros(len(df_day)).astype('float')
    
    pt_end = df_day[df_day['range-trail'] != 0].index[-1]
    pt_ini = df_day[df_day['range-trail'] != 0].index[0]
    
    pt_ini_date=df_es_02['date'][df_es_02[df_es_02['range-trail']!=0].index[0]].year*10000+ \
                df_es_02['date'][df_es_02[df_es_02['range-trail']!=0].index[0]].month*100+ \
                df_es_02['date'][df_es_02[df_es_02['range-trail']!=0].index[0]].day
    
    pt_end_date=df_es_02['date'][df_es_02[df_es_02['range-trail']!=0].index[-1]].year*10000+ \
                df_es_02['date'][df_es_02[df_es_02['range-trail']!=0].index[-1]].month*100+ \
                df_es_02['date'][df_es_02[df_es_02['range-trail']!=0].index[-1]].day
    
    pt_ini=locate_1st_day('D', pt_ini_date, df_int, 'datetime_EU')
    pt_end=locate_1st_day('D', pt_end_date, df_int, 'datetime_EU')
    
    print (pt_ini, pt_ini_date, '-', pt_end, pt_end_date)
    
    testing=pd.DataFrame(columns=['datetime_EU', 'open', 'max', 'min', 'close', 'long-stop', 'short-stop', 'long-exit', 'short-exit',
                                  'long-operation', 'short-operation', 'max-day', 'min-day', 'range-trail'])
    testing['open']= np.zeros(pt_end-pt_ini+1).astype('float')
    testing['max']= np.zeros(pt_end-pt_ini+1).astype('float')
    testing['min']= np.zeros(pt_end-pt_ini+1).astype('float')
    testing['close']= np.zeros(pt_end-pt_ini+1).astype('float')
    testing['long-stop']= np.zeros(pt_end-pt_ini+1).astype('float')
    testing['short-stop']= np.zeros(pt_end-pt_ini+1).astype('float')
    testing['long-exit']= np.zeros(pt_end-pt_ini+1).astype('float')
    testing['short-exit']= np.zeros(pt_end-pt_ini+1).astype('float')
    testing['long-operation']= np.zeros(pt_end-pt_ini+1).astype('float')
    testing['short-operation']= np.zeros(pt_end-pt_ini+1).astype('float')
    testing['max-day']= np.zeros(pt_end-pt_ini+1).astype('float')
    testing['min-day']= np.zeros(pt_end-pt_ini+1).astype('float')
    testing['range-trail']= np.zeros(pt_end-pt_ini+1).astype('float')
    
    # Define variables to use
    operation = 0                       # (1 in market, 0 out market)
    entry= 0.0
    long_stop = 0.0
    short_stop = 0.0
    long_exit = 0.0
    short_exit = 0.0
    long_operation = 0
    short_operation = 0
    maximum = 0.0
    minimum = 0.0
    market_exit = 0.0
    
    hour_ini=int(timetable_ini[0:2])                                  # initial values
    minu_ini=int(timetable_ini[3:])
    hour_end=int(timetable_end[0:2])
    minu_end=int(timetable_end[3:])
    tmtb_ini = hour_ini * 100 + minu_ini                              # integer value
    tmtb_end = hour_end * 100 + minu_end                              # integer value
    
    
    my_int_date_ant=0                                                  # define my date ant
    
    my_long_acc = 0.0                                                 # long result accumulator
    my_short_acc = 0.0                                               # short result accumulator
    j=0                                                                # counter for testing
    
    for ind in range(pt_ini, pt_end+1):                                  # the main loop
        
        my_int_date=df_int['datetime_EU'][ind].year * 10000 + df_int['datetime_EU'][ind].month * 100 + df_int['datetime_EU'][ind].day   # the day
        
        if (my_int_date > my_int_date_ant):                              # This is a new day 
            my_int_date_ant = my_int_date
            
            my_loc = locate_1st_day('D', my_int_date, df_day, 'date')      # locate the day in the df_day to take the range-trail
            range_trail = df_day['range-trail'][my_loc]                    # take the range-trail
            
        
        my_int_time=df_int['datetime_EU'][ind].hour * 100 + df_int['datetime_EU'][ind].minute     # the time
        
        # checking the bar time is >= than start time of timetable and < than end time of timetable  ==>> operation in market
        
        if ((my_int_time >= tmtb_ini) & (my_int_time < tmtb_end)):
            
            if ((my_int_time == tmtb_ini) & (operation == 0)):                 # The moment to start operation                    
                entry=df_int['open'][ind]
                long_stop = entry - range_trail
                short_stop = entry + range_trail
                long_exit = df_int['close'][ind]
                short_exit = df_int['close'][ind]
                long_operation = 1
                short_operation = 1
                maximum = df_int['max'][ind]
                minimum = df_int['min'][ind]
                operation = 1
            
            if (operation == 1):
                if (short_operation == 1):
                    short_exit=df_int['close'][ind]
                if (long_operation == 1):
                    long_exit=df_int['close'][ind]
                if ((df_int['max'][ind] > short_stop) & (short_operation==1)):        # if the price overcome the short_stop (up)
                    short_exit = short_stop
                    short_operation = 0
                if ((df_int['min'][ind] < long_stop) & (long_operation == 1)):         # if the price overcome the long_stop (down)
                    long_exit = long_stop
                    long_operation = 0
                if ((long_stop < df_int['max'][ind] - range_trail) & (long_operation==1)): # up long stop
                    long_stop = df_int['max'][ind] - range_trail
                if ((short_stop > df_int['min'][ind] + range_trail) & (short_operation==1)): # down short stop
                    short_stop = df_int['min'][ind] + range_trail
                if ((my_int_time == tmtb_end) & (long_operation == 1)):                     # long operation end because finish timetable
                    long_operation =0
                if ((my_int_time == tmtb_end) & (short_operation == 1)):                     # short operation end because finish timetable
                    short_operation =0
                if (df_int['max'][ind] > maximum):                                    # update maximum if it is the case
                    maximum = df_int['max'][ind]
                if (df_int['min'][ind] < minimum):                                    # update minimum if it is the case
                    minimum = df_int['min'][ind]
                market_exit = df_int['close'][ind]

        elif ((my_int_time == tmtb_end) & (operation == 1)):                            # timetable finish and save the result
            df_day['open'][my_loc]=entry
            df_day['long-close'][my_loc]=long_exit
            df_day['short-close'][my_loc]=short_exit
            df_day['close'][my_loc]=market_exit
            df_day['long-rst'][my_loc]=long_exit-entry
            df_day['short-rst'][my_loc]=entry-short_exit
            my_long_acc = my_long_acc + long_exit - entry
            df_day['long-acc'][my_loc]= my_long_acc
            my_short_acc = my_short_acc + entry - short_exit
            df_day['short-acc'][my_loc]= my_short_acc
            
            operation = 0                                                             # Re-init variables
            entry= 0.0
            long_stop = 0.0
            short_stop = 0.0
            long_exit = 0.0
            short_exit = 0.0
            long_operation = 0
            short_operation = 0
            maximum = 0.0
            minimum = 0.0
            market_exit = 0.0

        # Save tracking information to check the function works only for debuging and testing
        j+=1
        testing['datetime_EU'][j]=df_int['datetime_EU'][ind]
        testing['open'][j]= df_int['open'][ind]
        testing['max'][j]= df_int['max'][ind]
        testing['min'][j]= df_int['min'][ind]
        testing['close'][j]= df_int['close'][ind]
        testing['long-stop'][j]= long_stop
        testing['short-stop'][j]= short_stop
        testing['long-exit'][j]= long_exit
        testing['short-exit'][j]= short_exit
        testing['long-operation'][j]= long_operation
        testing['short-operation'][j]= short_operation
        testing['max-day'][j]= maximum
        testing['min-day'][j]= minimum
        testing['range-trail'][j]= range_trail

    return testing, df_day

In [13]:
my_test, my_day=market_operation (df_es_00, df_es_02, '09:00', '18:30')

104127 20140103 - 241683 20190906


In [17]:
my_test.to_csv('../Data/ES_test03.csv',index=False)

In [16]:
my_day.to_csv('../Data/ES_03.csv',index=False)



In [1]:
df_es_02

NameError: name 'df_es_02' is not defined

In [10]:
number=0.84
rnd=round(number, 1)
rnd

0.8